#h

In [ ]:
import torch
import torchvision

In [ ]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d heonh0/daynight-cityview

Dataset URL: https://www.kaggle.com/datasets/heonh0/daynight-cityview
License(s): CC0-1.0
100% 498M/499M [00:26<00:00, 22.8MB/s]
100% 499M/499M [00:27<00:00, 19.4MB/s]


In [ ]:
!unzip daynight-cityview.zip -d day_night

Archive:  daynight-cityview.zip
  inflating: day_night/day/0.jpg     
  inflating: day_night/day/1.jpg     
  inflating: day_night/day/10.jpg    
  inflating: day_night/day/100.jpg   
  inflating: day_night/day/101.jpg   
  inflating: day_night/day/102.jpg   
  inflating: day_night/day/103.jpg   
  inflating: day_night/day/104.jpg   
  inflating: day_night/day/105.jpg   
  inflating: day_night/day/106.jpg   
  inflating: day_night/day/107.jpg   
  inflating: day_night/day/108.jpg   
  inflating: day_night/day/109.jpg   
  inflating: day_night/day/11.jpg    
  inflating: day_night/day/110.jpg   
  inflating: day_night/day/111.jpg   
  inflating: day_night/day/112.jpg   
  inflating: day_night/day/113.jpg   
  inflating: day_night/day/114.jpg   
  inflating: day_night/day/115.jpg   
  inflating: day_night/day/116.jpg   
  inflating: day_night/day/117.jpg   
  inflating: day_night/day/118.jpg   
  inflating: day_night/day/119.jpg   
  inflating: day_night/day/12.jpg    
  inflating: day_n

In [ ]:
device="cuda" if torch.cuda.is_available else "cpu"
device

'cuda'

In [ ]:
import torchvision.transforms as transforms

transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
])

In [ ]:
dataset=torchvision.datasets.ImageFolder('day_night',transform=transform)

In [ ]:
dataloader=torch.utils.data.DataLoader(dataset,batch_size=8,shuffle=True)

In [ ]:
import torch
import torch.nn as nn

class Gen(nn.Module):
    def __init__(self, input_shape, output_shape):
        super().__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(input_shape, 64, kernel_size=7, stride=1, padding=3, padding_mode="reflect"),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),
        )

        self.block_2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1, padding_mode="reflect"),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),

            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1, padding_mode="reflect"),
            nn.InstanceNorm2d(256),
            nn.ReLU(inplace=True),
        )

        self.block_3 = nn.Sequential(
            *[
                nn.Sequential(
                    nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, padding_mode="reflect"),
                    nn.InstanceNorm2d(256),
                    nn.ReLU(inplace=True),

                    nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, padding_mode="reflect"),
                    nn.InstanceNorm2d(256),
                )
                for _ in range(9)
            ]
        )

        self.block_4 = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),
        )

        self.last = nn.Sequential(
            nn.Conv2d(64, output_shape, kernel_size=7, stride=1, padding=3, padding_mode="reflect"),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.block_1(x)
        x = self.block_2(x)
        for block in self.block_3:
            x = block(x) + x
        x = self.block_4(x)
        return self.last(x)

In [ ]:
discriminator=torchvision.models.resnet152(pretrained=True)

In [ ]:
discriminator.fc = nn.Sequential(
    nn.Linear(2048, 128),
    nn.ReLU(),
    nn.Linear(128, 1)
)

discriminator

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
gen_D=Gen(3,3)
gen_N=Gen(3,3)
dis_D=discriminator
dis_N=discriminator

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

Sun Jan 12 10:57:28 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
gen_D=gen_D.to(device)
gen_N=gen_N.to(device)
dis_D=dis_D.to(device)
dis_N=dis_N.to(device)

In [ ]:
D_optim=torch.optim.Adam(list(dis_D.parameters())+list(dis_N.parameters()), lr=0.002, betas=(0.5, 0.999))
G_optim=torch.optim.Adam(list(gen_D.parameters())+list(gen_N.parameters()), lr=0.002, betas=(0.5, 0.999))

/usr/local/lib/python3.10/dist-packages/torch/_compile.py:32: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  return disable_fn(*args, **kwargs)


In [ ]:
loss_fn1=nn.MSELoss()
loss_fn2=nn.L1Loss()

In [ ]:
g_scaler=torch.amp.GradScaler()
d_scaler=torch.amp.GradScaler()

In [ ]:
schedular=torch.optim.lr_scheduler.StepLR(G_optim, step_size=10, gamma=0.1)

In [ ]:
dataset.classes

['day', 'night']

In [ ]:
epochs=75

for epoch in range(epochs):
    for idx, (images, labels) in enumerate(dataloader):
        day, night = images[labels == 0], images[labels == 1]
        day, night = day.to(device), night.to(device)

        # Train Discriminators
        with torch.cuda.amp.autocast():
            gen_day = gen_D(night)
            gen_night = gen_N(day)

            d_real_day = dis_D(day)
            d_real_night = dis_N(night)

            d_fake_day = dis_D(gen_night.detach())
            d_fake_night = dis_N(gen_day.detach())

            d_loss_day = loss_fn1(d_real_day, torch.ones_like(d_real_day)) + \
                         loss_fn1(d_fake_day, torch.zeros_like(d_fake_day))
            d_loss_night = loss_fn1(d_real_night, torch.ones_like(d_real_night)) + \
                           loss_fn1(d_fake_night, torch.zeros_like(d_fake_night))

            d_loss = (d_loss_day + d_loss_night) / 2

        D_optim.zero_grad()
        d_scaler.scale(d_loss).backward()
        d_scaler.step(D_optim)
        d_scaler.update()

        # Train Generators
        with torch.cuda.amp.autocast():
            d_fake_day = dis_D(gen_night)
            d_fake_night = dis_N(gen_day)

            g_loss_day = loss_fn1(d_fake_day, torch.ones_like(d_fake_day))
            g_loss_night = loss_fn1(d_fake_night, torch.ones_like(d_fake_night))

            cycle_loss = loss_fn2(gen_D(gen_N(day)), day) + loss_fn2(gen_N(gen_D(night)), night)
            g_loss = (g_loss_day + g_loss_night) / 2 + 10 * cycle_loss

        G_optim.zero_grad()
        g_scaler.scale(g_loss).backward()
        g_scaler.step(G_optim)
        g_scaler.update()

        if idx % 45 == 0:
            print(f"Epoch [{epoch}/{epochs}] Batch {idx}, D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")
            if gen_day.numel() > 0:  # Ensure the tensor is not empty
                torchvision.utils.save_image(gen_day, f"/content/hel{epoch}_{idx}.png")
            else:
                print(f"Skipping save: gen_day is empty at Epoch {epoch}, Batch {idx}")


<ipython-input-26-b95fd4fd35fb>:9: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
<ipython-input-26-b95fd4fd35fb>:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch [0/75] Batch 0, D Loss: 1.0797122716903687, G Loss: 9.795147895812988
Epoch [0/75] Batch 45, D Loss: 0.46281710267066956, G Loss: 4.028853416442871
Epoch [0/75] Batch 90, D Loss: 0.4403151273727417, G Loss: 2.9019274711608887
Epoch [1/75] Batch 0, D Loss: 0.3176029920578003, G Loss: 3.752971649169922
Epoch [1/75] Batch 45, D Loss: 0.3688632845878601, G Loss: 2.821852684020996
Epoch [1/75] Batch 90, D Loss: 0.7599372267723083, G Loss: 3.5416688919067383
Epoch [2/75] Batch 0, D Loss: 0.44666528701782227, G Loss: 3.0893912315368652
Epoch [2/75] Batch 45, D Loss: 0.3844597339630127, G Loss: 3.323289394378662
Epoch [2/75] Batch 90, D Loss: 0.19283129274845123, G Loss: 2.8004987239837646
Epoch [3/75] Batch 0, D Loss: 0.4189014434814453, G Loss: 2.85498046875
Epoch [3/75] Batch 45, D Loss: 0.2517908811569214, G Loss: 3.1330325603485107
Epoch [3/75] Batch 90, D Loss: nan, G Loss: nan
Skipping save: gen_day is empty at Epoch 3, Batch 90
Epoch [4/75] Batch 0, D Loss: 0.28279659152030945, G

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import requests
from io import BytesIO

# Load pre-trained CycleGAN model for night-to-day conversion
model_url = "https://t4.ftcdn.net/jpg/10/59/41/33/360_F_1059413360_p0PAcBZPUPe96XerKVIy3kpZ14kjwUTA.jpg"

# Download model weights
model_path = "cycle_gan_night2day.pth"
if not torch.cuda.is_available():
    print("Warning: GPU not found. The process will be slower on CPU.")

# Define the transformation to prepare input images
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Define function to load and preprocess image from web URL
def load_image_from_url(url):
    response = requests.get(url)
    image = Image.open(BytesIO(response.content)).convert("RGB")
    return image

# Load an example night image from a web URL
night_image_url = "https://example.com/night_image.jpg"  # Replace with your image URL
night_image = load_image_from_url(night_image_url)
input_tensor = transform(night_image).unsqueeze(0)  # Add batch dimension

# Load and apply the model
class NightToDayModel(torch.nn.Module):
    # A placeholder model definition, this would be replaced by a real model in practice.
    def forward(self, x):
        # This should represent the forward pass of the night2day conversion.
        return x  # Dummy pass for example; real implementation uses trained CycleGAN.

# Initialize and apply the transformation model
model = NightToDayModel()
model.eval()  # Set to evaluation mode

# Perform image transformation
with torch.no_grad():
    output_tensor = model(input_tensor)

# Post-process and visualize the output
output_image = transforms.ToPILImage()(output_tensor.squeeze().cpu()).convert("RGB")
output_image.show()  # Show the transformed image
output_image.save("day_converted_image.jpg")  # Save the transformed image
